<a href="https://colab.research.google.com/github/creativesar/Rock-paper-scissors/blob/main/Rock_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import os

# Colors for terminal text
class Colors:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    CYAN = '\033[96m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

# Game elements with emoji
CHOICES = {
    "rock": "🪨",
    "paper": "📄",
    "scissors": "✂️"
}

# Power-ups that player can earn
POWER_UPS = {
    "double": "⚡ Double Points",
    "block": "🛡️ Block Loss",
    "peek": "👁️ See Computer's Move"
}

def clear_screen():
    """Clear the terminal screen."""
    os.system('cls' if os.name == 'nt' else 'clear')

def get_user_choice(power_ups):
    """Get the user's choice with power-up option."""
    print(f"{Colors.YELLOW}Choose your move:{Colors.RESET}")

    for i, (choice, emoji) in enumerate(CHOICES.items(), 1):
        print(f"{i}. {choice.capitalize()} {emoji}")

    # Show power-ups if available
    if any(power_ups.values()):
        print(f"\n{Colors.PURPLE}Power-ups available:{Colors.RESET}")
        for i, (power, name) in enumerate(POWER_UPS.items(), 4):
            if power_ups[power]:
                print(f"{i}. {name}")

    while True:
        choice = input(f"\n{Colors.BOLD}Your choice: {Colors.RESET}").lower()

        # Handle numeric input
        if choice.isdigit():
            num = int(choice)
            if 1 <= num <= 3:
                return list(CHOICES.keys())[num-1], None
            elif num == 4 and power_ups["double"]:
                return None, "double"
            elif num == 5 and power_ups["block"]:
                return None, "block"
            elif num == 6 and power_ups["peek"]:
                return None, "peek"

        # Handle text input
        elif choice in CHOICES:
            return choice, None
        elif choice in POWER_UPS and power_ups[choice]:
            return None, choice

        print(f"{Colors.RED}Invalid choice. Try again!{Colors.RESET}")

def play_game(scores, streak, power_ups):
    """Play a single round with special features."""
    clear_screen()
    print(f"{Colors.BOLD}{Colors.PURPLE}✨ ROCK PAPER SCISSORS SPECIAL ✨{Colors.RESET}")
    print(f"Score: You {scores['user']} - {scores['computer']} Computer (Streak: {streak})")

    # Handle peek power-up first
    computer_choice = random.choice(list(CHOICES.keys()))
    if power_ups["peek"]:
        use_peek = input(f"\n{Colors.CYAN}Use your Peek power-up to see computer's move? (y/n): {Colors.RESET}").lower() == 'y'
        if use_peek:
            print(f"{Colors.CYAN}Computer will choose: {computer_choice.upper()} {CHOICES[computer_choice]}{Colors.RESET}")
            power_ups["peek"] = False

    # Get user choice
    user_choice, power_up = get_user_choice(power_ups)

    # Handle power-ups
    double_points = False
    block_loss = False

    if power_up == "double":
        print(f"{Colors.YELLOW}Double Points activated!{Colors.RESET}")
        double_points = True
        power_ups["double"] = False
        user_choice, _ = get_user_choice({k: False for k in power_ups})

    elif power_up == "block":
        print(f"{Colors.YELLOW}Block Loss activated!{Colors.RESET}")
        block_loss = True
        power_ups["block"] = False
        user_choice, _ = get_user_choice({k: False for k in power_ups})

    # Show choices
    print(f"\n{Colors.BLUE}You chose: {user_choice.upper()} {CHOICES[user_choice]}{Colors.RESET}")

    # Computer choice with suspense
    print(f"{Colors.RED}Computer is choosing", end="")
    for _ in range(3):
        time.sleep(0.3)
        print(".", end="", flush=True)
    print(f" {computer_choice.upper()} {CHOICES[computer_choice]}{Colors.RESET}")

    # Determine winner
    if user_choice == computer_choice:
        result = "tie"
        message = f"{Colors.YELLOW}It's a tie!{Colors.RESET}"
        streak = 0
    elif ((user_choice == "rock" and computer_choice == "scissors") or
          (user_choice == "paper" and computer_choice == "rock") or
          (user_choice == "scissors" and computer_choice == "paper")):
        result = "user"
        message = f"{Colors.GREEN}You win!{Colors.RESET}"
        streak = streak + 1 if streak >= 0 else 1

        # Award power-up on streak
        if streak == 3:
            available = [p for p, active in power_ups.items() if not active]
            if available:
                power_up = random.choice(available)
                power_ups[power_up] = True
                print(f"{Colors.PURPLE}🎁 3-Win Streak! You earned: {POWER_UPS[power_up]}{Colors.RESET}")
    else:
        if block_loss:
            result = "tie"
            message = f"{Colors.CYAN}Loss blocked by your shield!{Colors.RESET}"
            streak = 0
        else:
            result = "computer"
            message = f"{Colors.RED}Computer wins!{Colors.RESET}"
            streak = streak - 1 if streak <= 0 else -1

    # Update score
    if result != "tie":
        if result == "user" and double_points:
            scores[result] += 2
            print(f"{Colors.YELLOW}+2 points from Double Points!{Colors.RESET}")
        else:
            scores[result] += 1

    print(f"\n{message}")

    return streak

def main():
    """Main game function with special features."""
    scores = {"user": 0, "computer": 0}
    streak = 0  # Positive for player streak, negative for computer streak
    power_ups = {"double": False, "block": False, "peek": False}

    # Give initial power-up
    power_ups["peek"] = True
    print(f"{Colors.CYAN}You start with: {POWER_UPS['peek']}{Colors.RESET}")
    time.sleep(1.5)

    while True:
        streak = play_game(scores, streak, power_ups)

        print(f"\n{Colors.YELLOW}Play again? (yes/no){Colors.RESET}")
        if input(f"{Colors.BOLD}> {Colors.RESET}").lower() != "yes":
            clear_screen()
            print(f"{Colors.PURPLE}===== FINAL SCORE ====={Colors.RESET}")
            print(f"You: {scores['user']} | Computer: {scores['computer']}")

            if scores['user'] > scores['computer']:
                print(f"{Colors.GREEN}🏆 You are the champion!{Colors.RESET}")
            elif scores['computer'] > scores['user']:
                print(f"{Colors.RED}💔 Better luck next time!{Colors.RESET}")
            else:
                print(f"{Colors.YELLOW}🤝 It's a tie!{Colors.RESET}")

            print(f"{Colors.BLUE}Thanks for playing!{Colors.RESET}")
            break

if __name__ == "__main__":
    main()

You start with: 👁️ See Computer's Move
✨ ROCK PAPER SCISSORS SPECIAL ✨
Score: You 0 - 0 Computer (Streak: 0)

Use your Peek power-up to see computer's move? (y/n): y
Computer will choose: ROCK 🪨
Choose your move:
1. Rock 🪨
2. Paper 📄
3. Scissors ✂️

Your choice: 2

You chose: PAPER 📄
Computer is choosing... ROCK 🪨

You win!

Play again? (yes/no)
